In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator,load_img
import tensorflow as tf

In [2]:
train_gen=ImageDataGenerator(rescale=1./255)
train_data=train_gen.flow_from_directory('../input/satellite-image-classification/data',
                                         target_size=(224,224))

Found 5631 images belonging to 4 classes.


# MOBILENETV2

In [3]:
# Transfer learning 
from keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input 

model = Sequential()
base_model = MobileNetV2(input_shape = [224, 224, 3], weights = 'imagenet', include_top = False)

2022-07-23 06:49:42.633143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 06:49:42.737788: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 06:49:42.738750: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 06:49:42.740146: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

9420800/9406464 [==============================] - 0s 0us/step


In [4]:
from tensorflow.keras.layers import GlobalAveragePooling2D
base_model.trainable = True  

for i in base_model.layers[:100]: 
    base_model.trainable = False
    
pool = GlobalAveragePooling2D()
mid_layer = Dense(100,activation = 'relu')
final_1 = Dense(4,activation = 'softmax')

model = Sequential([base_model, mid_layer,pool, final_1])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
dense (Dense)                (None, 7, 7, 100)         128100    
_________________________________________________________________
global_average_pooling2d (Gl (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
Total params: 2,386,488
Trainable params: 128,504
Non-trainable params: 2,257,984
_________________________________________________________________


In [5]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = 'accuracy')

In [6]:
model.fit(train_data, batch_size=32, epochs=10)

2022-07-23 06:49:47.897976: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-07-23 06:49:51.119592: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


176/176 [==============================] - 52s 250ms/step - loss: 0.1569 - accuracy: 0.9515
Epoch 2/10
176/176 [==============================] - 14s 80ms/step - loss: 0.0508 - accuracy: 0.9849
Epoch 3/10
176/176 [==============================] - 13s 75ms/step - loss: 0.0370 - accuracy: 0.9879
Epoch 4/10
176/176 [==============================] - 13s 73ms/step - loss: 0.0266 - accuracy: 0.9913
Epoch 5/10
176/176 [==============================] - 13s 74ms/step - loss: 0.0204 - accuracy: 0.9940
Epoch 6/10
176/176 [==============================] - 12s 70ms/step - loss: 0.0231 - accuracy: 0.9917
Epoch 7/10
176/176 [==============================] - 13s 74ms/step - loss: 0.0218 - accuracy: 0.9934
Epoch 8/10
176/176 [==============================] - 13s 75ms/step - loss: 0.0160 - accuracy: 0.9941
Epoch 9/10
176/176 [==============================] - 14s 82ms/step - loss: 0.0124 - accuracy: 0.9954
Epoch 10/10
176/176 [==============================] - 13s 74ms/step - loss: 0.0120 - accura